In [1]:
import pandas as pd


In [2]:
df = pd.read_csv(r"C:\Users\pranu\Downloads\spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Within Temptation,Dog Days,Waking in the dead of night \r\nHold onto thi...
1,Lucky Dube,Lovers In A Dangerous Time,He built this wall around himself \r\nMany ye...
2,The Beatles,I Will,Who knows how long I've loved you \r\nYou kno...
3,Toto,House Of The Rising Sun,There is a house in New Orleans \r\nThey call...
4,Erasure,Star,Star \r\nWe go waiting for the stars \r\nTo ...
5,Billy Joel,Honesty,If you search for tenderness \r\nIt isn't har...
6,Great Big Sea,Captain Wedderburn,A noblemen's fair daughter \r\nCame down a na...
7,Kinks,Muswell Hillbilly,Well I said goodbye to Rosie Rooke this mornin...
8,Dean Martin,Just For Fun,Love me love me say you love me \r\nJust for ...
9,Kyla,Let The Love Begin,"Look at us, ain't it funny \r\nIs it just beg..."


In [9]:
df['text'][0]

"Waking in the dead of night  \r\nHold onto this life  \r\nRunning to what's wrong or right  \r\nSeems to be a thin line  \r\nFighting for a wasted love  \r\nFighting to survive  \r\nOh won't you please forget  \r\nWon't you please forget  \r\n  \r\nThese are the dog days  \r\nThese are the dog days  \r\nThey'll never go away  \r\nThese are the dog days  \r\nThese are the dog days  \r\nThey'll never go away  \r\n  \r\n1, 2, 3, 4  \r\nWhat are you waiting for?  \r\n1, 2, 3, 4  \r\nWhat are you waiting for?  \r\nThese are the dog days  \r\nThese are the dog days  \r\n  \r\nWaking in the dead of night  \r\nHold onto this life  \r\nRunning to what's wrong or right  \r\nSeems to be a thin line  \r\nFighting for a wasted love  \r\nFighting to survive  \r\nOh won't you please forget  \r\nWon't you please forget  \r\n  \r\nThese are the dog days  \r\nThese are the dog days  \r\nThey'll never go away  \r\nThese are the dog days  \r\nThese are the dog days  \r\nThey'll never go away  \r\n  \r\n1

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [18]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pranu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
import nltk 
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [20]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [23]:
similarity[0]

array([1.        , 0.02829402, 0.04194053, ..., 0.00364421, 0.09008016,
       0.1440038 ])

In [24]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [32]:
def recommendation(song_name):
    try:
        idx = df[df['song'] == song_name].index[0]
        distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
        # Rest of your code for recommendations
        # ...
        songs = []
        for m_id in distances[1:21]:
            songs.append(df.iloc[m_id[0]].song)
        return songs
    except IndexError:
        return "Song not found in the DataFrame"


In [33]:
recommendation('Crying Over You')

'Song not found in the DataFrame'

In [34]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))